In [1]:
import dask
import numpy as np

In [2]:
# from dask.distributed import Client

# client = Client(n_workers=8)
# client

In [3]:
import dask.dataframe as dd

rankings = dd.read_csv(
    "data/AMPLab/1node/rankings/*.csv",
    header=None,
    names=["pageURL", "pageRank", "avgDuration"],
)
rankings.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 3 entries, pageURL to avgDuration
dtypes: object(1), int64(2)

In [4]:
%%time
query_1 = rankings[rankings.pageRank > 1000]
len(query_1)

CPU times: user 17.9 s, sys: 3.06 s, total: 21 s
Wall time: 16.8 s


6552

In [4]:
uservisits = dd.read_csv(
    "data/AMPLab/1node/uservisits/*.csv",
    header=None,
    names=["sourceIP", "destinationURL", "visitDate", "adRevenue", "UserAgent", "cCode", "lCode", "searchWord", "duration"],
    parse_dates=["visitDate"],
)
uservisits.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, sourceIP to duration
dtypes: datetime64[ns](1), object(6), float64(1), int64(1)

In [6]:
%%time
query_2 = (
    uservisits
        .groupby(uservisits.sourceIP.str.slice(0, 7))
        .adRevenue
        .sum()
)
query_2.compute()

CPU times: user 8min 50s, sys: 37.8 s, total: 9min 28s
Wall time: 6min 30s


sourceIP
1.0.129    4.167757
1.0.133    7.449813
1.0.163    2.376591
1.0.173    7.524194
1.0.196    3.103560
             ...   
7.0.4.9    0.336308
1.7.3.5    0.018638
1.8.7.4    0.662783
2.7.0.7    0.550923
3.4.4.9    0.768892
Name: adRevenue, Length: 359235, dtype: float64

In [5]:
%%time
query_3 = (rankings
    .merge(uservisits[
              (uservisits.visitDate > "1980-01-01")
            & (uservisits.visitDate < "1980-04-01")
           ],
           left_on="pageURL",
           right_on="destinationURL",
           how="left",
    )
    .groupby("sourceIP")
    .agg({"pageRank": "mean", "adRevenue": "sum"})
    .sort_values("adRevenue", ascending=False)
    .head(1)
)
query_3.compute()

ignoring exception in ensure_cleanup_on_exception
Traceback (most recent call last):
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/dask/dataframe/shuffle.py", line 953, in ensure_cleanup_on_exception
    yield
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/dask/dataframe/shuffle.py", line 968, in shuffle_group_3
    p.append(d, fsync=True)
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/partd/encode.py", line 25, in append
    self.partd.append(data, **kwargs)
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/partd/buffer.py", line 45, in append
    self.flush(keys)
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/partd/buffer.py", line 99, in flush
    self.slow.append(dict(zip(keys, self.fast.get(keys))))
  File "/home/filonov/playground/dask/.venv/lib/python3.10/site-packages/partd/file.py", line 42, in append
    f.write(v)
OSError: [Errno 28] No space left o

OSError: [Errno 28] No space left on device